<a href="https://colab.research.google.com/github/Vidit122/Mini-Project/blob/main/FinalMobileNetV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DATA PREPROCESSING**

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, QuantileTransformer
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Reshape, Conv2D, DepthwiseConv2D,
    BatchNormalization, ReLU, GlobalAveragePooling2D,
    Bidirectional, GRU, Dense, Dropout, Concatenate
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
df = pd.read_csv("5G_NIDD_FULL_9CLASS_MIXED.csv", low_memory=False)

print(df.shape)
print(df.columns[:10])
print(df["Label"].value_counts())

print(df.shape)
print(df.columns[:10])
print(df["Label"].value_counts())

(1693627, 51)
Index(['Max', 'AckDat', 'DstLoss', 'dDSb', 'Sum', 'Mean', 'SrcTCPBase', 'sDSb',
       'dTtl', 'TotBytes'],
      dtype='object')
Label
Benign         477737
UDPFlood1      467717
UDPFlood2      286197
Goldeneye1      93803
Goldeneye2      93650
SYNFlood1       44636
Torshammer1     38120
Torshammer2     31669
Slowloris1      31015
ICMPFlood1      18279
ICMPFlood2      14452
SYNFlood2       14108
Slowloris2      12656
TCPConnect2     11653
TCPConnect1     11645
SYNScan2        11526
SYNScan1        11450
UDPScan2        10305
UDPScan1        10043
SSH1             1608
SSH2             1358
Name: count, dtype: int64
(1693627, 51)
Index(['Max', 'AckDat', 'DstLoss', 'dDSb', 'Sum', 'Mean', 'SrcTCPBase', 'sDSb',
       'dTtl', 'TotBytes'],
      dtype='object')
Label
Benign         477737
UDPFlood1      467717
UDPFlood2      286197
Goldeneye1      93803
Goldeneye2      93650
SYNFlood1       44636
Torshammer1     38120
Torshammer2     31669
Slowloris1      31015
ICMPFlood1    

In [ ]:
# ALWAYS start from df
X = df.drop(columns=["Label"])
y = df["Label"]

# Force numeric
X = X.apply(pd.to_numeric, errors="coerce")
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(X.mean(), inplace=True)

print("X shape BEFORE slicing:", X.shape)
 # MUST be > 36 columns

X shape BEFORE slicing: (1693627, 50)


In [ ]:
# X = X.iloc[:, :36]
# print("X shape AFTER slicing:", X.shape)


In [ ]:
label_map = {
    # HTTP floods
    "Goldeneye1": "HTTP_Flood",
    "Goldeneye2": "HTTP_Flood",
    "Torshammer1": "HTTP_Flood",
    "Torshammer2": "HTTP_Flood",

    # Slow DoS
    "Slowloris1": "Slowrate_DoS",
    "Slowloris2": "Slowrate_DoS",

    # UDP flood
    "UDPFlood1": "UDP_Flood",
    "UDPFlood2": "UDP_Flood",

    # SYN flood
    "SYNFlood1": "SYN_Flood",
    "SYNFlood2": "SYN_Flood",

    # ICMP flood
    "ICMPFlood1": "ICMP_Flood",
    "ICMPFlood2": "ICMP_Flood",

    # Scans
    "UDPScan1": "UDP_Scan",
    "UDPScan2": "UDP_Scan",

    "SYNScan1": "SYN_Scan",
    "SYNScan2": "SYN_Scan",

    "TCPConnect1": "TCP_Connect_Scan",
    "TCPConnect2": "TCP_Connect_Scan",

    # Benign (already correct)
    "Benign": "Benign",

    # SSH (DROP — not used)
    "SSH1": None,
    "SSH2": None
}

In [ ]:
df["Label"] = df["Label"].map(label_map)

# Remove rows mapped to None (SSH etc.)
df = df.dropna(subset=["Label"])

In [ ]:
print(sorted(df["Label"].unique()))
print("Number of classes:", df["Label"].nunique())

['Benign', 'HTTP_Flood', 'ICMP_Flood', 'SYN_Flood', 'SYN_Scan', 'Slowrate_DoS', 'TCP_Connect_Scan', 'UDP_Flood', 'UDP_Scan']
Number of classes: 9


In [ ]:
# Features & labels
X = df.drop(columns=["Label"])
y = df["Label"]

X = X.apply(pd.to_numeric, errors="coerce")
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(X.mean(), inplace=True)

# Convert to numeric
X = X.apply(pd.to_numeric, errors="coerce")

# Remove inf first
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaNs BEFORE QT
X.fillna(X.mean(), inplace=True)


# EXACT 36 features (fixed, no filtering later)
X = X.iloc[:, :36]

# Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_enc, 9)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42, stratify=y_enc
)


# Quantile transform (paper style)
qt = QuantileTransformer(
    n_quantiles=1000,
    output_distribution="normal",
    random_state=42
)
X_train = qt.fit_transform(X_train)
X_test  = qt.transform(X_test)

X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)
X_test  = np.nan_to_num(X_test,  nan=0.0, posinf=0.0, neginf=0.0)

# Reshape
X_train = X_train.reshape(-1, 36, 1)
X_test  = X_test.reshape(-1, 36, 1)


/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1634: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


In [ ]:
print("NaNs in X:", np.isnan(X_train).sum())
print("Infs in X:", np.isinf(X_test).sum())
print("y unique sums:", np.unique(y_train.sum(axis=1)))

NaNs in X: 0
Infs in X: 0
y unique sums: [1.]


In [ ]:
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical
# import numpy as np

# X = df.drop(columns=["Label"])
# y = df["Label"]

# le = LabelEncoder()
# y_encoded = le.fit_transform(y)

# NUM_CLASSES = len(np.unique(y_encoded))
# y_onehot = to_categorical(y_encoded, NUM_CLASSES)

# print("Final NUM_CLASSES:", NUM_CLASSES)

Final NUM_CLASSES: 9


In [ ]:
# # ALWAYS start from df
# X = df.drop(columns=["Label"])
# y = df["Label"]

# # Force numeric
# X = X.apply(pd.to_numeric, errors="coerce")
# X.replace([np.inf, -np.inf], np.nan, inplace=True)
# X.fillna(X.mean(), inplace=True)

# print("X shape BEFORE slicing:", X.shape)
#  # MUST be > 36 columns

X shape BEFORE slicing: (1690661, 50)


In [ ]:
# # Replace inf → nan
# X.replace([np.inf, -np.inf], np.nan, inplace=True)

# # Fill nan with column mean
# X.fillna(X.mean(), inplace=True)

# # Final check
# print("NaNs:", X.isna().sum().sum())
# print("Infs:", np.isinf(X.values).sum())

NaNs: 10143966
Infs: 0


In [ ]:
# # MUST be done on X BEFORE split & scaling
# X = X.iloc[:, :36]
# print("X shape after slicing:", X.shape)

X shape after slicing: (1690661, 36)


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X,
#     y_onehot,
#     test_size=0.2,
#     random_state=42,
#     stratify=y_encoded
# )

In [ ]:
# qt = QuantileTransformer(
#     n_quantiles=1000,
#     output_distribution="normal",
#     random_state=42
# )

# X_train = qt.fit_transform(X_train)
# X_test  = qt.transform(X_test)

/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1634: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


In [ ]:
# print("X_train shape before reshape:", X_train.shape)
# print("X_test shape before reshape:", X_test.shape)

X_train shape before reshape: (1352528, 36)
X_test shape before reshape: (338133, 36)


In [ ]:
# X_train = X_train.reshape(-1, 36, 1, 1)
# X_test  = X_test.reshape(-1, 36, 1, 1)


# print("Train shape:", X_train.shape)
# print("Test shape:", X_test.shape)

Train shape: (1352528, 36, 1, 1)
Test shape: (338133, 36, 1, 1)


In [ ]:
# from sklearn.utils.class_weight import compute_class_weight
# import numpy as np

# class_weights = compute_class_weight(
#     class_weight="balanced",
#     classes=np.unique(y_encoded),
#     y=y_encoded
# )

# class_weights = dict(enumerate(class_weights))
# print(class_weights)

{0: np.float64(0.39321053680628093), 1: np.float64(0.7302509785424706), 2: np.float64(5.739244820574447), 3: np.float64(3.1977941955301343), 4: np.float64(8.175975897554936), 5: np.float64(4.301509519411559), 6: np.float64(8.062976316517393), 7: np.float64(0.2491679717079431), 8: np.float64(9.231925605574121)}


**MOBILENET V1 MODEL**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D,
    BatchNormalization, ReLU,
    GlobalAveragePooling2D, Dense, LayerNormalization
)
from tensorflow.keras.optimizers import Adam

In [ ]:
def depthwise_separable_conv(x, filters):
    x = DepthwiseConv2D((3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters, (1,1), padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

In [ ]:
def depthwise_separable_conv(x, filters):
    x = DepthwiseConv2D((3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters, (1,1), padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

In [ ]:
def MobileNetV1_5G(input_shape=(36,1,1), num_classes=8):
    inputs = Input(shape=input_shape)

    # Initial Conv
    x = Conv2D(32, (3,3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv(x, 64)
    x = depthwise_separable_conv(x, 128)

    # Global pooling
    x = GlobalAveragePooling2D()(x)

    # 🔑 NUMERICAL STABILITY (THIS FIXES NaN LOSS)
    x = LayerNormalization()(x)

    # Classifier
    outputs = Dense(num_classes, activation="softmax")(x)

    return Model(inputs, outputs)

In [ ]:
from tensorflow.keras.optimizers import Adam

model = MobileNetV1_5G(
    input_shape=(36,1,1),
    num_classes=9
)

model.compile(
    optimizer=Adam(learning_rate=1e-3),   # LOWER LR = 1e-4  #better lr = 5e-5
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 36, 1, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 36, 1, 32)      │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 36, 1, 32)      │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 36, 1, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d                │ (None, 36, 1, 32)      │           320 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 36, 1, 32)      │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 36, 1, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 36, 1, 64)      │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 36, 1, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 36, 1, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 36, 1, 64)      │           640 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 36, 1, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 36, 1, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 36, 1, 128)     │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 36, 1, 128)     │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 36, 1, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 128)            │           256 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,409 (56.29 KB)

 Trainable params: 13,769 (53.79 KB)

 Non-trainable params: 640 (2.50 KB)

In [ ]:
# import numpy as np

# X_train[np.isinf(X_train)] = np.nan
# X_test[np.isinf(X_test)] = np.nan

# X_train = np.nan_to_num(X_train)
# X_test = np.nan_to_num(X_test)

# print("NaNs:", np.isnan(X_train).sum())
# print("Infs:", np.isinf(X_train).sum())

NaNs: 0
Infs: 0


In [ ]:
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=64,
    validation_split=0.1,
    verbose=1
)


Epoch 1/15
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 109s 5ms/step - accuracy: 0.6228 - loss: 0.8580 - val_accuracy: 0.6425 - val_loss: 0.7145
Epoch 2/15
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 109s 5ms/step - accuracy: 0.6523 - loss: 0.6620 - val_accuracy: 0.6415 - val_loss: 0.7472
Epoch 3/15
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 101s 5ms/step - accuracy: 0.6546 - loss: 0.6406 - val_accuracy: 0.6380 - val_loss: 0.6724
Epoch 4/15
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 100s 5ms/step - accuracy: 0.6569 - loss: 0.6291 - val_accuracy: 0.6662 - val_loss: 0.6396
Epoch 5/15
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 101s 5ms/step - accuracy: 0.6582 - loss: 0.6198 - val_accuracy: 0.6637 - val_loss: 0.7016
Epoch 6/15
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 100s 5ms/step - accuracy: 0.6597 - loss: 0.6125 - val_accuracy: 0.6506 - val_loss: 0.6852
Epoch 7/15
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 142s 5ms/step - accuracy: 0.6602 - loss: 0.6118 - val_accuracy: 0.6397 - val_loss: 0.6884
Epoch 8/15
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 100s 5ms/step - ac

In [ ]:
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=64,
    validation_split=0.1,
    verbose=1
)


Epoch 1/10
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 113s 5ms/step - accuracy: 0.7043 - loss: 0.6276 - val_accuracy: 0.7151 - val_loss: 0.5615
Epoch 2/10
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 95s 4ms/step - accuracy: 0.7125 - loss: 0.5631 - val_accuracy: 0.7164 - val_loss: 0.5950
Epoch 3/10
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 96s 5ms/step - accuracy: 0.7142 - loss: 0.5519 - val_accuracy: 0.7094 - val_loss: 0.5787
Epoch 4/10
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 96s 5ms/step - accuracy: 0.7141 - loss: 0.5474 - val_accuracy: 0.7158 - val_loss: 0.5399
Epoch 5/10
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 95s 5ms/step - accuracy: 0.7149 - loss: 0.5438 - val_accuracy: 0.7164 - val_loss: 0.5351
Epoch 6/10
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 97s 5ms/step - accuracy: 0.7153 - loss: 0.5402 - val_accuracy: 0.7177 - val_loss: 0.5345
Epoch 7/10
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 101s 5ms/step - accuracy: 0.7154 - loss: 0.5387 - val_accuracy: 0.7171 - val_loss: 0.5374
Epoch 8/10
21134/21134 ━━━━━━━━━━━━━━━━━━━━ 99s 5ms/step - accuracy

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)

10567/10567 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - accuracy: 0.7185 - loss: 0.5291
Test Accuracy: 0.7180872559547424


In [ ]:
import numpy as np

labels, counts = np.unique(y_enc, return_counts=True)
for l, c in zip(le.classes_, counts):
    print(l, c)

Benign 477737
HTTP_Flood 257242
ICMP_Flood 32731
SYN_Flood 58744
SYN_Scan 22976
Slowrate_DoS 43671
TCP_Connect_Scan 23298
UDP_Flood 753914
UDP_Scan 20348


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, y_pred, target_names=le.classes_))

10567/10567 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step
                  precision    recall  f1-score   support

          Benign       0.50      0.59      0.55     95548
      HTTP_Flood       0.97      0.80      0.88     51448
      ICMP_Flood       0.66      0.05      0.10      6546
       SYN_Flood       0.91      0.18      0.30     11749
        SYN_Scan       1.00      0.88      0.93      4595
    Slowrate_DoS       0.90      0.30      0.45      8734
TCP_Connect_Scan       0.95      0.87      0.91      4660
       UDP_Flood       0.78      0.85      0.81    150783
        UDP_Scan       1.00      0.77      0.87      4070

        accuracy                           0.72    338133
       macro avg       0.85      0.59      0.64    338133
    weighted avg       0.74      0.72      0.71    338133

